In [1]:
import json
import gzip
import os
from datetime import timedelta, date
import pandas as pd

In [2]:
from paper.config import *

by_dir

'/mnt/storage2/Git/jooble-ir-2018-paper/Data/by'

In [5]:
list_json = []

start_date = date(2017, 1, 1)
# end_date = date(2017, 1, 30)
end_date = date(2017, 1, 2)

date_range = pd.date_range(start_date, end_date)
    
#looping through all days from 2017_01_1 to 2017_01_30 with step = 1 day
for single_date in date_range:
    date_ = single_date.strftime("%Y-%m-%d")
        
    #!!!!CHANGE PATH!!!!!!
    file_job = os.path.join(by_dir, 'by_job_{}.json.gz'.format(date_))
    file_job_text = os.path.join(by_dir, 'by_job_text_{}.json.gz'.format(date_))

    #reading two files simuntaneously
    document_job, document_job_text = [],[]
    with gzip.open(file_job, "rb") as f1, gzip.open(file_job_text, "rb") as f2: 
        for line1, line2 in zip(f1, f2):
            vacation_job = json.loads(line1)
            vacation_job_text = json.loads(line2)
            document_job.append(vacation_job)
            document_job_text.append(vacation_job_text)

        #creating a list of dictionaries
        for i in range(len(document_job)):
            for j in range(len(document_job_text)):
                if document_job[i]['id'] == document_job_text[j]['id_job']:
                    data = {}
                    data['id'] = document_job[i]['id']
                    data['lang_text'] = document_job[i]['lang_text']
                    data['lang_title'] = document_job[i]['lang_title']
                    data['company_name'] = document_job[i]['company_name']
                    data['title'] = document_job[i]['title']
                    data['title_normalized'] = document_job[i]['title_normalized']
                    data['salary_val1'] = document_job[i]['salary_val1']
                    data['salary_val2'] = document_job[i]['salary_val2']
                    data['company_name'] = document_job[i]['company_name']
                    data['url'] = document_job[i]['url']
                    
                    data['text'] = document_job_text[j]['text']
                    list_json.append(data)
                    
print(len(list_json))

707


## Make Index update via `updater` service

In [8]:
from paper.utils import post_request as post
from paper.index import delete_all

delete_all()

update_json = {'docs': list_json}

post(UPDATER_PORT, UPDATER_PATH, update_json)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
from paper.index import get_docs

get_docs([1,2])

[{'company_name': '',
  'id': '-6951509096940958272',
  'lang_text': 'ru',
  'lang_title': 'ru',
  'salary_val1': '',
  'salary_val2': '',
  'text': 'Срочно отрисовать не сложный ч/б логотип в вектор.',
  'title': 'Дизайнер логотипов. Срочно отрисовать не сложный ч/б логотип в вектор (удаленно)',
  'title_normalized': 'дизайнер логотипов срочно отрисовать не сложный ч б логотип в вектор удаленно',
  'url': 'https://www.fl.ru/projects/3085095/srochno-otrisovat-ne-slojnyiy-ch_b-logotip-v-vektor.html?utm_source=joobleby&utm_medium=cpc&utm_campaign=jooble'},
 {'company_name': '',
  'id': '8709047890904426760',
  'lang_text': 'ru',
  'lang_title': 'ru',
  'salary_val1': '',
  'salary_val2': '',
  'text': 'курсовая работа по Экономика фирмы \nдисциплина - Экономика фирмы \nтема - вариант 2 в методичке "тема работы" \nколичество страниц - 25-30 \nоригинальность -60% (отчет с сайта антиплагиат.ру приложить) \nшрифт - 14 pt \nкрайний срок сдачи - 03.01.2017 до 22 по МСК \nдополнительная информа